## Generate meta data of genes and samples for human_gene_v2.2.h5

In [2]:
import os
param_json_file = "/g/data/yr31/zs2131/tasks/2023/RNA_expr_net/GeneRAIN/jsons/exp2_GPT_Binning_By_Gene.param_config.json"
os.environ['PARAM_JSON_FILE'] = param_json_file

from train.common_params_funs import get_gene_symbols_filt_on_z_dup
genes_with_good_mean_zscore_nondup = get_gene_symbols_filt_on_z_dup()

Run in training mode!
param_json_file is /g/data/yr31/zs2131/tasks/2023/RNA_expr_net/anal/training/exp20/exp20_GPT_uniform_bin.param_config.json
Parameter of GENE_EMB_NAME not found in the input json file, use default value of gene2vec!
Parameter of TOTAL_NUMBER_OF_DATASETS not found in the input json file, use default value of 5!
Parameter of DATASET_TO_GET_FOR_MIXED_DATASET not found in the input json file, use default value of None!
Parameter of FINETUNE_TO_RECONSTRUCT_EXPR_OF_ALL_GENES not found in the input json file, use default value of False!
Parameter of OUTPUT_ATTENTIONS not found in the input json file, use default value of False!
Parameter of OUTPUT_HIDDEN_STATES not found in the input json file, use default value of False!
Parameter of ONLY_USE_PERTURBED_GENE_TO_PREDICT not found in the input json file, use default value of False!
Parameter of LEARN_ON_ZERO_EXPR_GENES not found in the input json file, use default value of False!
Parameter of OUTPUT_PARAMETER_HIST_TO_TENSOB

/home/561/zs2131/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Reading gene_to_idx_path /g/data/yr31/zs2131/tasks/2023/RNA_expr_net/DeepGeneNet/data/embedding/gene2vec_gene_to_idx.json!
Reading gene_to_idx_path /g/data/yr31/zs2131/tasks/2023/RNA_expr_net/DeepGeneNet/data/embedding/gene2vec_gene_to_idx.json!


In [ ]:
import h5py
import torch
import numpy as np
from torch.utils.data import Dataset
import os
import pandas as pd
from utils.json_utils import JsonUtils
ju = JsonUtils()

from utils.config_loader import Config

config = Config()
proj_path = config.proj_path

h5_file_path = proj_path + "/data/external/ARCHS/human_gene_v2.2.h5"
h5_file = h5py.File(h5_file_path, "r")
gene_symbols = h5_file['meta']['genes']['symbol'][()]
sample_size = h5_file['data']['expression'].shape[1]
meta_data = h5_file['meta']['samples']

os.chdir(proj_path + "/data/external/ARCHS/")
# Extract metadata from hdf5 file to pandas DataFrame
def output_meta(meta_data, output_file):
    df_meta_data = pd.DataFrame()
    for key in meta_data.keys():
        df_meta_data[key] = meta_data[key][()]
    
    # Convert bytes to string
    df_meta_data = df_meta_data.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
    
    # Save metadata DataFrame to a tsv file
    df_meta_data.to_csv(output_file, sep='\t', index=False)
output_meta(h5_file['meta']['genes'], output_file=f"{h5_file_path}.genes_meta.tsv")
output_meta(h5_file['meta']['samples'], output_file=f"{h5_file_path}.samples_meta.tsv")